In [16]:
import numpy as np
import random
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 6)

In [12]:
# Пример трюков с массивами numpy
a = np.array( [
        [10, 11, 12, 13, 14],
        [20, 21, 22, 23, 24],
        [30, 31, 32, 33, 34]] )
bs = np.array( [0, 1, 2, 0, 1])
b = a[bs, np.arange(5)]
b

array([10, 21, 32, 13, 24])

In [69]:
def rf_adjust_scalar( n, x ):
    if np.isscalar( x ):
        return np.full( n, x )
    else:
        return x

def test_rf_adjust_scalar():
    print( "Test rf_adjust_scalar" )
    a = rf_adjust_scalar( 5, 0.5 )
    print( a )
    print( rf_adjust_scalar( 5, a ) )
    print( rf_adjust_scalar( 5, 1 ) )
    
def rf_adjust_scalars( n, array ):
    adjusted = []
    for i, x in enumerate( array ):
        adjusted.append( rf_adjust_scalar( n, x ) )
    return np.array( adjusted )

def test_rf_adjust_scalars():
    print( "Test rf_adjust_scalars" )
    print( rf_adjust_scalars( 5, [0, 1, 2] ) )
    print( rf_adjust_scalars( 5, [0.5, 0.2] ) )
    print( rf_adjust_scalars( 5, [0.5, rf_adjust_scalar( 5, 0.3 )] ) )

def rf_random( n ):
    return np.random.random( n )

def test_rf_random():
    print( "Test rf_random" )
    print( rf_random( 5 ) )

def rf_discrete( n, distribution ):
    p_array = rf_adjust_scalars( n, distribution )
    p_sum = sum( p_array )
    r = np.random.random( n ) * p_sum
    threshold = np.zeros( n );
    result = np.zeros( n, dtype='int' )
    for i, p in enumerate( p_array ):
        result[r >= threshold] = i
        threshold = threshold + p
    return result

def test_rf_discrete():
    print( "Test rf_discrete" )
    print( rf_discrete( 10, [0.5, 0.7] ) )
    p = rf_random( 10 )
    print( p )
    print( rf_discrete( 10, [1 - p, p] ) )

# ToDo: Должен быть режимом rf_discrete
def rf_binary( n, one_probability ):
    return rf_discrete( n, [1 - one_probability, one_probability] )

def rf_select( indexes, sources ):
    n = len( indexes )
    adjusted = rf_adjust_scalars( n, sources );
    return np.array( adjusted )[np.array( indexes ), np.arange( n )]

def test_rf_select():
    print( "Test rf_select" )
    print( rf_select( a, [1, 0, 0] ) )

# ToDo: Должен быть rf_pselect когда rf_binary будет режимом rf_discrete
def rf_dnoise( p, signal, noise ):
    is_noise = rf_binary( len( signal ), p )
    return rf_select( is_noise, [signal, noise] )

def rf_salt( p, a ):
    return rf_dnoise( p, a, 0 )

def rf_pepper( p, a ):
    return rf_dnoise( p, a, 1 )

def test():
    print( "Test begin" )
    test_rf_adjust_scalar()
    test_rf_adjust_scalars()
    test_rf_random()
    test_rf_discrete()
    test_rf_select()
    print( "Test end" )
    print()

test()


Test begin
Test rf_adjust_scalar
[ 0.5  0.5  0.5  0.5  0.5]
[ 0.5  0.5  0.5  0.5  0.5]
[1 1 1 1 1]
Test rf_adjust_scalars
[[0 0 0 0 0]
 [1 1 1 1 1]
 [2 2 2 2 2]]
[[ 0.5  0.5  0.5  0.5  0.5]
 [ 0.2  0.2  0.2  0.2  0.2]]
[[ 0.5  0.5  0.5  0.5  0.5]
 [ 0.3  0.3  0.3  0.3  0.3]]
Test rf_random
[ 0.24433769  0.09993543  0.3736128   0.02388577  0.60070982]
Test rf_discrete
[0 1 1 1 1 1 1 1 0 1]
[ 0.60925779  0.86391295  0.81875777  0.46739232  0.12346002  0.79729207
  0.46006776  0.88401523  0.23099059  0.28269298]
[0 1 1 0 0 1 1 1 0 1]
Test rf_select
[1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 0 1 0 1]
Test end



In [82]:
n = 30

#a = rf_discrete( n, [0.7, 0.2, 0.1] )
a = np.concatenate( ( np.full( 10, 0 ), np.full( 10, 1 ), np.full( 10, 2 ) ) )
print( "a:" )
print( a )
print()

b1 = rf_select( a, [1, 0, 0] )
b2 = rf_select( a, [1, 0, 0] )
b3 = rf_select( a, [1, 1, 0] )
b4 = rf_select( a, [0, 1, 0] )
b5 = rf_select( a, [0, 1, 1] )
b6 = rf_select( a, [0, 0, 1] )
b7 = rf_select( a, [0, 0, 1] )
print( "bi:" )
print( b1 )
print( b2 )
print( b3 )
print( b4 )
print( b5 )
print( b6 )
print( b7 )
print()

noise_p = rf_random( n ) / 2
print( "noise probability:" )
print( noise_p )
print()

c1 = b1
c2 = rf_salt( 0.1, rf_pepper( noise_p, b2 ) )
c3 = rf_select( rf_binary( n, noise_p ), [b3, rf_binary( n, 0.5 )] )
c4 = rf_dnoise( noise_p, b4, rf_binary( n, 0.5 ) )
c5 = rf_salt( 0.1, rf_pepper( 0.1, b5 ) )
c6 = rf_salt( noise_p, rf_pepper( 0.1, b6 ) )
c7 = rf_salt( noise_p, rf_pepper( 0.1, b7 ) )
print( "ci:" )
print( c1 )
print( c2 )
print( c3 )
print( c4 )
print( c5 )
print( c6 )
print( c7 )
print()


a:
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]

bi:
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]

noise probability:
[ 0.15631376  0.30820685  0.45294981  0.35140908  0.3453506   0.48344452
  0.38749496  0.4418051   0.08857166  0.26721266  0.24781506  0.15078051
  0.3176686   0.1023316   0.12819505  0.33963739  0.02672105  0.35738311
  0.16659158  0.07253333  0.22589134  0.06983124  0.12550797  0.42673208
  0.16567574  0.39524521  0.23720924  0.07568405  0.42441461  0.09458989]

ci:
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 